In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as f
import torchvision
from torchvision import transforms
import os
import csv
from skimage import io, transform
from PIL import Image

In [2]:
dataset_path = os.path.join(os.curdir, 'dataset')
with_mask_ds_path = os.path.join(dataset_path, 'with_mask')
without_mask_ds_path = os.path.join(dataset_path, 'without_mask')
csv_file = os.path.join(dataset_path, 'face-mask-detection-dataset.csv')

In [3]:
# Make a csv for dataset
if os.path.exists(csv_file):
    print("CSV File Found")
else:
    print("CSV Not Found")    
    print("Creating CSV File")
    with open('face-mask-detection-dataset.csv', mode='w') as f:
        col_names = ['image_name', 'label']
        writer = csv.DictWriter(f, fieldnames=col_names)
        writer.writeheader()
        for img in os.listdir(without_mask_ds_path):
            writer.writerow({'image_name': img, 'label': 0})
        for img in os.listdir(with_mask_ds_path):
            writer.writerow({'image_name': img, 'label': 1})
        f.close()

CSV File Found


In [4]:
class MaskDetectionDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.csv_file = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self):
        return len(self.csv_file)
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img_name = self.csv_file.iloc[idx, 0]
        label = self.csv_file.iloc[idx, 1]
        if (label==0):
            img_path = os.path.join(self.root_dir, 'without_mask', img_name)
            img = io.imread(img_path)
        else:
            img_path = os.path.join(self.root_dir, 'with_mask', img_name)
            img = io.imread(img_path)
        img = Image.fromarray(img)
        if self.transform:
            img = self.transform(img)
        sample = {
            'image': img,
            'label': label
        }
        return sample

In [5]:
# Prepare Image by transformation

augmentation_transform = transforms.Compose({
    transforms.Resize((300,300)),
    transforms.ToTensor(),
})

dataset = MaskDetectionDataset(csv_file='./dataset/face-mask-detection-dataset.csv', root_dir=dataset_path, transform=augmentation_transform)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

lr = 1e-4
epochs = 20
batch_size = 32

In [7]:
print(len(dataset))

820


In [8]:
# Splitting dataset
train_set, test_set = torch.utils.data.random_split(dataset, [750, 70])
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [9]:
for batch_idx, sample in enumerate(train_loader):
    print(batch_idx)
    print(sample['image'])
    print(sample['label'])
    break

0
tensor([[[[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 4.4879e-01,
           3.4874e-01, 2.9060e-01],
          [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 4.5652e-01,
           3.4251e-01, 2.7992e-01],
          [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.2871e-01,
           3.3111e-01, 2.9964e-01],
          ...,
          [9.8850e-01, 9.9647e-01, 1.0000e+00,  ..., 8.7277e-01,
           8.8050e-01, 9.3414e-01],
          [9.8850e-01, 9.9647e-01, 1.0000e+00,  ..., 8.8779e-01,
           8.9224e-01, 9.2547e-01],
          [9.8850e-01, 9.9647e-01, 1.0000e+00,  ..., 8.8870e-01,
           8.9664e-01, 9.1184e-01]],

         [[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 4.2289e-01,
           3.1390e-01, 2.6421e-01],
          [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 4.0515e-01,
           3.2169e-01, 2.9663e-01],
          [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 4.1451e-01,
           2.6418e-01, 2.9177e-01],
          ...,
          [1.0000e+00, 1.0000e+00, 9.9935e-01,  ..., 8.10